<div class="licence">
<span>Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat</span>
<span><img src="media/inria-25-alpha.png" /></span>
</div>

# `git` tutorial (6)

## synchronisations entre repos

In [ ]:
# ce sera toujours notre façon de commencer
[ -f scripts/helpers.sh ] && source scripts/helpers.sh

## architecture décentralisée

* on a vu plusieurs architectures permettant de créer des workflows
* comme par exemple

![](media/archi-star.png)

* dans ce schéma 
  * les boites sont des repos
  * les liens correspondent à des synchronisations entre repos  

## notre exemple

pour cette partie :

* nous repartons d'un repo quasiment vide `repo-alice`
* que l'on va *cloner* dans `repo-cloned`
* puis les modifier (créer des commits)
* et les synchroniser entre eux 

In [ ]:
cd $TOP

# on recommence un autre repo plus simple; à nouveau
# je nettoie complètement ce qu'on a pu faire précédemment
if [ -d repo-alice ]; then
    echo "on repart d'un directory vide"
    rm -rf repo-alice repo-cloned fake-github.git repo-bob
fi

# on le crée
mkdir repo-alice

# on va dedans
cd repo-alice

In [ ]:
# si nécessaire, on se place dans le repo git
[ -d repo-alice ] && cd repo-alice

pwd

## création

In [ ]:
$SCRIPTS/do populate-repo-alice

In [ ]:
git l

## le point

In [ ]:
# vous devez avoir 2 commits
git l


In [ ]:
# avec deux branches `master` et `devel`
# on est sur la branch devel
git branch

## pour les besoins du cours

* en général, les repos sont créés sur des **machines distinctes**
  * typiquement : un repo local + un sur github
* techniquement, pas obligatoire
  * **pour les besoins du cours**,  
    nous allons créer nos repos **localement** 

* les mécanismes de copie/synchronisation sont exactement identiques

In [ ]:
# on utilisera ce répertoire $TOP/repo-cloned
# pour simuler un deuxième repo
# 
# on verra plus tard qu'en pratique deux personnes
# qui travaillent ensemble passent par un troisième 
# repo sur github, mais pour l'instant on veut 
# seulement bien illustrer les fonctions `fetch` et `pull` 

if [ -d $TOP/repo-cloned ]; then
    rm -rf $TOP/repo-cloned
fi

## `git clone` sert à dupliquer un repo

In [ ]:
cd $TOP

git clone repo-alice/.git repo-cloned

In [ ]:
cd $TOP/repo-cloned

ls

le clone va contenir :

* **les commits**, c'est-à-dire la partie `bare`
* les fichiers présents dans le **commit courant**
* mais par contre l'index n'est **pas concerné**
  * si on avait eu des modifications pendantes dans `repo-alice`
  * que ce soit dans l'index ou les fichiers
  * ils **n'auraient pas** été copiés

In [ ]:
git status

In [ ]:
git l

* en réalité dans un `git clone`
  * du coté source, seul le ***bare repo*** est lu
  * les fichiers et l'index ne sont **pas du tout regardés**
  * bien souvent d'ailleurs la source est sur une infra comme github
  * et dans ce cas la source n'est **que** un *bare repo*
  
* les fonctions de synchro entre repos
  * ne concernent en réalité  
    que la partie 'bare repo' des deux cotés

  * en général les fichiers et index **ne sont pas concernés**  
    par les synchros entre repo

  * qui ne font principalement que transférer des commits  
    (et mettre à jour des branches)
    
* sauf dans le cas de `pull` 
  * qui en réalité fait `fetch` + `merge`
  * et du coup `merge` peut être amené à toucher les fichiers

à noter surtout :

* **pas de hiérarchie** entre les repos
  * la source et le clone sont des **pairs** - pas de *master*/*slave* 
* la seule différence bien sûr ce sont les droits d'accès
  * en gros c'est **chacun chez soi**
  * si vous avez les droits d'accès (linux, windows, macOS)  
    sur un repo vous pouvez écrire dedans

autre remarque :

* les SHA1 des commits **sont préservés**
* la copie se fait quasiment à l'octet près
* ce qui permet un mode de réplication incrémental
  * si je copie un gros repo un lundi
  * et que je tire depuis ce repo mardi
  * on va efficacement calculer ce qu'il est réellement utile de transférer

In [ ]:
cd $TOP/repo-alice
git l -1

In [ ]:
cd $TOP/repo-cloned
git l -1

## `git clone` en vrai 

l'usage le plus fréquent consiste à dupliquer un repo qui est publié sur `github`

par exemple ce cours est sur
https://github.com/flotpython/gittutorial/

![](media/github-clone.png)

tapez 

    git clone 
    
et faites 'coller' avec Control-V

    git clone git@github.com:flotpython/gittutorial.git
    
vous pouvez alors faire

    cd gittutorial
    git log --oneline
    ls

## les fonctions de synchronisation

en plus de `clone`, les fonctions de synchronisation sont :

* `fetch`
* `pull`
* `push`

**passif**

* `fetch`  injecte des commits distants dans le repo local **sans impact local**

**actif**

* `push`: injecte des commits locaux dans le repo distant
* `pull` = `fetch`+`merge`


## la notion de *remote*

avant de voir en détail les fonctions de synchro,  
nous devons voir la notion de *remote*

un *remote*, c'est 

* essentiellement **un nom** symbolique
* qui nous permet de faire facilement référence à un **autre repo**
* i.e. plutôt que de retaper **son URL** à chaque fois

## les *remote*s

dans notre clone, notez la présence d'un *remote* appelé `origin`

In [ ]:
# nous sommes dans le clone
pwd

In [ ]:
# pour lister les remotes connus
git remote

au moment du `clone`, git a créé pour nous ce *remote* avec le nom prédéfini `origin`, qui désigne le repo d'où on a cloné

In [ ]:
# en version bavarde on voit à quoi correspond le remote 
git remote -v

pour info : comme pour les branches, on peut facilement ajouter, renommer, etc. les *remote*s

## branches et remotes

un repo git est *self-contained*

* toutes les références (branche et remote) sont des **objets locaux**
* on peut toujours travailler sans connexion réseau

pour résumer, deux notions très différentes

* la branche désigne un point dans les commits (forcément locaux)
* le remote est simplement une **référence** vers un autre repo
  * c'est juste un nom, un alias, vers un autre repo
* ainsi par exemple
  * on peut sans souci créer un remote vers un repo inexistant
  * c'est seulement quand on s'en sert - via fetch/push/pull - qu'on se rendra compte du problème

## branches et remotes - suite

* comme un repo est *self-contained*
* il conserve **localement** la trace des branches distantes
* voyez par exemple les branches `origin/master` et `origin/devel`


In [ ]:
# depuis le clone, on voit un nouveau type 
# de référence, comme par exemple origin/master

git l --all

qu'on pourrait paraphraser comme ceci :

* du point de vue du repo `repo-cloned`
* il y a dans le repo distant `origin` (donc, `repo-alice`)
* une branche `master` 
* qui pointe vers ce commit

## branches et remotes - suite

cette information **n'est pas** garantie d'être 100% à jour !

on va le voir tout de suite : 

* si je crée dans `my-first-repo` un commit
  * c'est une opération **strictement locale**
* le clone `repo-cloned` n'en n'est pas informé immédiatement
  * à nouveau, c'est du pair à pair / chacun chez soi
* il le sera essentiellement s'il fait un `fetch`

remarque :

* on peut configurer énormément de choses; par exemple
  * décider de synchoniser deux repos après chaque commit
* mais c'est de l'ordre du confort
  * ce sont des **opérations élémentaires distinctes**

## synchro - fetch

notre scénario

* créer un nouveau commit dans le repo originel `my-first-repo`
* observer les deux repos à ce stade
* déclencher un `fetch` depuis `repo-cloned`
* observer les deux repos à ce stade

## fetch (1) - créer un commit à l'origine

In [ ]:
cd $TOP/repo-alice

In [ ]:
git l


In [ ]:
$SCRIPTS/do first-commit-in-alice

In [ ]:
git l

## avant le `fetch`

In [ ]:
cd $TOP/repo-alice
git l

In [ ]:
cd $TOP/repo-cloned
git l

## fetch (2) - utiliser `fetch` depuis le clone

In [ ]:
# je retourne sur le clone
cd $TOP/repo-cloned

# on va chercher les commits nouveaux
# en faisant --all on va sur tous les remote connus
# ici on n'en a qu'un, c'est origin = initial
git fetch --all

## après le `fetch`

In [ ]:
# le repo initial

cd $TOP/repo-alice
git l

In [ ]:
# le repo après fetch
cd $TOP/repo-cloned

# si je ne précise pas --all
# on part comme toujours de HEAD
git l 

In [ ]:
# mais si j'ajoute --all:

git l --all

remarquez :

* nous voyons un nouveau commit !
* `origin/devel` est mis à jour
* ainsi d'ailleurs que `origin/HEAD`
* remarque / digression : 
  * pas de branch locale `master`
  * on pourrait cependant faire
  * `git checkout master` 
  * on ne va pas le faire 

## mettre à jour les références locales

à ce stade, pour me mettre à jour par rapport au repo distant, je peux

* merger `origin/devel` dans `devel`
* ce qui ferait avancer `devel` d'un cran 
* noter que c'est un merge `fast-forward`
* et donc, **pas de création** de commit

In [ ]:
git merge origin/devel

In [ ]:
git l

## `pull = fetch + merge`

* c'est exactement le propos de `pull`
* que d'automatiser ce genre d'opérations
* en une seule passe

plusieurs formes

* `git pull origin master` 
  * met à jour `origin/master` depuis le remote `origin`
  * et le merge dans `master`
* `git pull origin master:devel` 
  * met à jour `origin/master` et le merge dans `devel`

## `pull` et raccourcis

* on peut même encore raccourcir, grâce à la configuration
* en partant du nom de la branche courante, ici `devel`
* qui a été configurée lorsqu'on a fait `clone`

In [ ]:
# git config permet de lire
# un attribut dans la config

# ceci est le défaut pour le
# premier argument à git pull
git config branch.devel.remote

In [ ]:
# pareil pour le deuxième argument

git config branch.devel.merge

du coup je peux encore enlever des arguments

* `git pull origin`
  * revient à `git pull origin devel`
* `git pull` 
  * revient `git pull origin devel`

## `pull` sur plusieurs branches

enfin on peut indiquer plusieurs branches sur le même remote

* `git pull origin master:master devel:devel` 

peut être utile quand on sait qu'on suit toujours plusieurs branches

## résumé sur *fetch* et *merge* et *pull*

pour résumer jusqu'ici :

* `fetch`
  * avec `git fetch` on sait aller chercher 
    les commits présents dans les autres repos

  * je recommande de faire `git fetch --all` 
    car la commande est principalement inoffensive

  * les outils genre SourceTree font cela périodiquement par défaut

* `merge` 
  * on peut ensuite merger ces commits
  * exactement comme si on les avait créés localement
  * notamment vis-à-vis des *fast-forward* 
  
* `pull` 
  * permet de faire les deux phases en une fois

## dans l'autre sens: `push`


* le modèle étant symétrique (pair à pair)
* à première vue, on se dit que le *push* 
* c-à-d propager des commits locaux vers un repo ditant
* **devrait** être l'**exact symétrique** du *pull*


* en pratique ce n'est **pas le cas**
* on travaille dans une **autre repo**
  * le plus souvent un *bare repo*

## dissymétrie

la dissymétrie est liée à la résolution de conflits :

* lors d'un `pull`, il y a un humain qui peut résoudre les conflits, revenir en arrière, etc..
* lors d'un `push`, ce **n'est pas forcément le cas** (pensez `github`)

c'est pourquoi :

* l'opération de `push` est effectivement l'inverse de `pull`
* on recopie à distance les commits qui n'y sont pas encore
* et on merge dans la branche distante
* mais c'est **limité à des *fast-foward***
* de cette façon on élimine la possibilité de conflits
* même si ça peut paraître trop conservatoire

## push et droits d'accès

notez aussi que, bien entendu, lors d'un push :

* il faut bien sûr les **droits d'écriture** dans le repo distant

dans le cas d'un repo distant sur github (gitlab, ...)

* il faut faire une démarche particulière pour obtenir ce droit
* **ou bien** se créer un *fork* (c'est leur principale raison d'être)
* on reparlera de tout ça

## cas d'usage

en pratique le `push` est utilisé pour

* exposer un travail sur un repo public - toujours *bare*
* de façon à ce que les collaborateurs  
  puissent alors l'importer dans leur repo avec un `pull`
  
d'ailleurs 

* `git push` **se plaint** si on essaie de pousser  
  vers un **repo qui n'est pas *bare***


## pour expérimenter

* nous allons revoir du coup notre setup
* on conserve `repo-alice`
* on détruit `repo-cloned`
* on crée à la place un repo *bare* qui s'appelle `fake-github.git`
* on va voir tout de suite pourquoi ce nom en `.git`

In [ ]:
cd $TOP
rm -rf repo-cloned fake-github.git
git clone --bare repo-alice fake-github.git

## un *bare* repo

ce qui nous donne l'occasion de voir à quoi ça ressemble

In [ ]:
cd $TOP

# le contenu d'un repo bare
ls fake-github.git

In [ ]:
# est proche du contenu d'un .git
# dans un repo 'normal'

ls repo-alice/.git

enfin disons, surtout en ce qui concerne 

* `config`
* `objects`: c'est là que sont rangés les commits et leurs contenus
* `refs`: c'est là que sont rangées les branches

on a l'habitude d'appeler les *bare* repo  
avec un nom en `.git` pour indiquer leur type (juste une convention)

## un push simple

scénario #1 : un push qui se passe bien

* je crée un commit dans le repo original
* je le pousse sur le faux github

## push simple

In [ ]:
cd $TOP/repo-alice

$SCRIPTS/do commit-in-initial-for-simple-push

git l

In [ ]:
cd $TOP/fake-github.git
git l

## préparation

quelques précautions sont à prendre toutefois pour pouvoir pousser


* la syntaxe de `push` est similaire à celle de pull
* il va donc nous falloir un `remote` 

## il nous faut un `remote`

In [ ]:
# on avait bien créé un remote plus haut mais
# c'était dans `repo-bob`

cd $TOP/repo-alice

# mais ici dans repo-alice on ne connait aucun remote
git remote

In [ ]:
# il va donc nous falloir définir un remote à la main
# et cette fois plutôt que de l'appeler `origin` on va l'appeler `github` 
# ce sera beaucoup plus parlant pour nous

git remote add github $TOP/fake-github.git

In [ ]:
git remote -v

## avant le push

In [ ]:
# la situation dans initial
# on a 4 commits

cd $TOP/repo-alice

git l --all

In [ ]:
# et dans le clone
# seulement 3 commits

cd $TOP/fake-github.git

git l --all

## mon premier push

In [ ]:
# on se met dans le repo initial

cd $TOP/repo-alice

# la syntaxe de push est voisine de celle de pull
# on pourrait faire simplement
#
# git push github devel


# cela dit je recommande par sécurité 
# d'éviter toute ambiüité 
# et de faire explicitement
#
git push github devel:devel

## après le push

In [ ]:
# ainsi après le push 
# les deux repos sont 
# en phase



cd $TOP/repo-alice
git l 

In [ ]:
# remarque un peu digressive
# voyez que github
# ne connait aucun remote
# c'est bien le cas dans la vraie vie
# car ce n'est jamais github 
# qui pousse ou qui tire

cd $TOP/fake-github.git
git l 

## résumé

nous avons à présent tous les éléments pour construire  
le plus simple travail collaboratif :

* alice crée un repo local sur son laptop
* elle travaille un moment seule, crée des commits
* elle publie son repo sur gihub
  * création d'un repo via l'interface web
  * ajout d'un remote dans son repo local
  * push
* bob peut alors créer un clone sur son laptop
  * et si alice lui donne les droits d'écriture  
    (toujours via l'interface web de github)

  * alors bob peut pousser lui aussi son travail

![](media/archi-gh-3.png)

In [ ]:
# on repart d'un repo tout simple avec seulement deux commits 
# pour ne pas encombrer inutilement l'affichage

# on nettoie
cd $TOP
rm -rf repo-alice fake-github.git repo-bob

# recrée repo-alice avec deux commits
cd $TOP
mkdir repo-alice
cd repo-alice
$SCRIPTS/do populate-repo-alice > /dev/null

# on crée un repo bare qui remplace github
# pour faire le proxy entre les deux acteurs
cd $TOP
git clone --bare repo-alice fake-github.git

# on clone le faux github dans repo-bob
cd $TOP
git clone fake-github.git repo-bob

les trois repos sont en phase

In [ ]:
cd $TOP/repo-alice
git l

In [ ]:
cd $TOP/fake-github.git
git l

In [ ]:
cd $TOP/repo-bob
git l

## un push compliqué

scénario #2 :  
un push qui ne se passe pas bien

* un cas **très fréquent** en pratique
* et qui génère pas mal de frustration 

* deux personnes se mettent à travailler  
* sur des sujets différents
* mais en même temps  
  (i.e. en partant du même commit)

* ils commitent chacun de leur coté

## le `push` qui tue (1)

In [ ]:
# alice avance de son coté

cd $TOP/repo-alice
$SCRIPTS/do commit-alice

git l

In [ ]:
# bob aussi

cd $TOP/repo-bob
$SCRIPTS/do commit-bob

git l

## il faut un remote pour pousser

à ce stade les deux acteurs vont vouloir pousser leur travail sur github  
pour ça il leur faut un remote

In [ ]:
cd $TOP/repo-alice
git remote add github $TOP/fake-github.git
git remote

cd $TOP/repo-bob
git remote add github $TOP/fake-github.git
git remote

## le `push` qui tue (2)

le premier des deux qui veut pousser sur github n'a aucun problème

In [ ]:
cd $TOP/repo-alice

git push github devel:devel 

In [ ]:
git l

In [ ]:
cd $TOP/fake-github.git
git l

## le `push` qui tue (3)

* bob veut pousser lui aussi
* c'est à ce stade que ça coince
* car ce push crée implique un merge
* qui n'**est pas *fast-forward***
* car en effet le graphe des commits ressemble à ceci

```
devel pour alice et github        devel pour bob
                     ↳   A      B   ↵
                           \   /
                             C2
                             |
                             C1
```

* un `push` de la part de bob 
* revient donc à merger `B` au dessus de `A`
* qui comme on le voit ne sont pas comparables

## le `push` qui tue (4)

In [ ]:
# si bob essaie de pousser à ce stade, c'est refusé
# car le merge, qui n'est pas fast-forward, impliquerait
# la création d'un nouveau commit, ce qui
# est risqué à distance 
#
# notez bien qu'ici les deux modifications de alice et bob
# sont indépendantes et peuvent être mergées sans conflit !

cd $TOP/repo-bob
git push github devel

## tirer avant de pousser

In [ ]:
# pour s'en sortir il suffit que Bob commence par tirer
# et c'est en tirant qu'on va créer le commit qui merge les deux travaux
# 
# pour des raisons sordides liées au fait qu'on est dans un notebook
# je lui passe l'option --no-edit

cd $TOP/repo-bob
git pull --no-edit github devel 

In [ ]:
git l

## le `push` ne tue plus

```
                                M   ←  devel pour bob
devel pour alice et github    /   \    
                        ↳   A      B  
                              \   /
                                C2
                                |
                                C1
```


pour Bob à présent, le fait de pousser `devel` sur `github` est redevenu un *fast-forward*, il peut pousser

In [ ]:
cd $TOP/repo-bob
git push github devel

In [ ]:

git l

In [ ]:
# et alice peut tirer
cd $TOP/repo-alice
git pull github devel

In [ ]:
git l

## résumé

* on copie un repo avec `clone`

* on va chercher les mises à jour de manière non intrusive avec `fetch`

* souvent on veut ensuite les appliquer localement
  * c'est à dire `fetch` puis `merge` -> c'est le propos de `pull`

  
* `push` est en gros l'inverse de `pull` 
  * sauf qu'il faut les droits d'accès
  * et qu'on ne contrôle pas les `remote` distants
  * c'est pourquoi il n'y a pas le symétrique de `fetch` 

  
* attention lors d'un `push`
  * à bien vérifier qu'il n'y a pas besoin de tirer d'abord
  * voir aussi `rebase` qu'on verra plus tard